# Prueba de Hipotesis <br>
Vamos a utilizar la aproximacion binomial con la prueba de hipotesis $ H~A $ Y $ H O $

In [209]:
import math
def normal_cdf(x, mu =0,sigma=1):
    return(1+ math.erf((x-mu)/math.sqrt(2)/sigma)) /2

In [210]:
normal_cdf(0.1)

0.539827837277029

In [211]:
def inverse_normal_cdf(p , mu=0,sigma=1,tolerance=0.00001):
    #busca aproximar la inversa usando busqueda binaria

    #si no es estandar, computa estandar y reescala
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p,tolerance=tolerance)
    low_z, low_p = -10.0,0 #normal cdf(-10) es muy cerca a 0
    hi_z , hi_p = 10.0,1    #normal cdf(10) es muy cerca a 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2 #consideren el punto medio
        print (mid_z)
        mid_p = normal_cdf(mid_z)   #y el valor de cdf aca
        print (mid_p)
        if mid_p < p:
            #puntomedio aun es muy bajo busca arriba de el
            print("here 1")
            low_z,low_p = mid_z,mid_p
        elif mid_p > p :
            #punto medio aun es muy alto , buscar abajo de el
            hi_z,hi_p = mid_z,mid_p
            print("here 2")
        else:
            break
        return mid_z


In [212]:
inverse_normal_cdf(0.5)

0.0
0.5


In [213]:
import math
def normal_aproximation_to_binomial(n,p):
    #Busca mu y sigma que corresponde a la binomial (n,p)
    mu = p * n
    sigma = math.sqrt(p * (1-p) * n)
    return mu, sigma

normal_aproximation_to_binomial(0.1,0.9)

(0.09000000000000001, 0.09486832980505137)

In [214]:
# La normal cdf_is_the_probability la variable es debajo del limite
normal_probability_below = normal_cdf

In [215]:
# Esta arriba el limite si eno pertenece por debajo del limite
def normal_probability_above(lo, mu=0,sigma=1):
    return 1 - normal_cdf(lo,mu,sigma)

In [216]:
# Esta entre si es menor que hi, pero no menor que lo
def normal_probability_between(lo,hi,mu=0,sigma = 1):
    return normal_cdf(hi,mu,sigma) - normal_cdf(lo,mu,sigma)
    

In [217]:
def  normal_probability_outside(lo,hi,mu=0,sigma=1):
    return 1 - normal_probability_between(lo,hi,mu,sigma)


In [218]:
def normal_upper_bound(probability, mu=0,sigma=1):
    # regresa z para cada P(Z<=Z) = probabilidad
    return inverse_normal_cdf(probability,mu,sigma)

In [219]:
def normal_lower_bound(probability,mu=0,sigma=1):
    #regresa z para cada P (Z>=Z) = probabilidad
    return inverse_normal_cdf(1-probability,mu,sigma)

In [220]:
def normal_two_sided_bounds(probability,mu=0,sigma=1):
    #regresa el simetrico ( cerca de la media) limite
    #que contiene la probabilidad especifica
    tail_probability = (1-probability) / 2
    #arriba de los limites debe tener la cola de probabilidad arriba de 
    upper_bound = normal_lower_bound(tail_probability,mu,sigma)
    #lower limite debe tener la cola de probabilidad abajo de
    lower_bound = normal_upper_bound(tail_probability,mu,sigma)

    return lower_bound, upper_bound

Ahora debemos de hacer nuestra prueba de hipotesis simple con una moneda donde haremos el intento  con mu = 0 , sigma = 0 y la aproximacion normal binomial esta dada por 1000 intentos con probabilidad 0.5

In [221]:
mu_0,sigma_0 = normal_aproximation_to_binomial(1000,0.5)

In [222]:
normal_two_sided_bounds(0.95,mu_0,sigma_0)

0.0
0.5
here 1
0.0
0.5
here 2


(500.0, 500.0)

Al parecer con el codigo del libro me arrojo algun especie de error, que raro. En fin continuare con algo que buscare en internet

# Prueba Z 
### Utilizando libreria stats

La fórmula anterior se puede entender como:Media de la muestra: la media de la población (es decir, la distancia entre ellos) dividida por una desviación estándar de la muestra, el resultado es que la distancia entre ellos es igual a varias veces la desviación estándar., Si es mayor que 2, es decir, la desviación excede 2 veces la desviación estándar, cae en el dominio de rechazo. La desviación estándar de esta distribución media viene dada por el teorema del límite central

Un ejemplo sencillo: muestra de cinco lotes de lecheTemperatura muy fríaSon $  - 0.547, -0.532, -0.548, -0.531, -0.535  $, y la temperatura del punto de congelación de la leche natural obedece a una distribución normal,El valor medio es -0,545 ℃, la desviación estándar es 0,008 ℃, Si se mezcla agua, la temperatura del punto de congelación aumentará y tenderá gradualmente a 0 ℃

In [223]:
import numpy as np
from scipy import stats


# Establezca la temperatura del punto de congelación de 5 lotes de leche muestreados
a = np.array([-0.547, -0.532, -0.548, -0.531, -0.535])

# Establecer la desviación estándar y media de la población
mean, std = -0.545, 0.008

# Calcule la media muestral
sample_mean = a.mean()

# Calcule el error estándar (desviación estándar de la población / número de población bajo el signo radical)
se = std / np.sqrt(len(a))

# Estadísticas Z
Z = (sample_mean - mean) / se

print("La estadística Z es {}".format(Z))

# Valor P
P = 2 * stats.norm.sf(abs(Z)) # Aquí stats.norm.sf () devuelve el área en el lado derecho de Z, por lo que * 2 es la suma de las áreas en los lados izquierdo y derecho de Z
 
print('El valor p es {}'.format(P))

La estadística Z es 1.788854381999821
El valor p es 0.07363827012030438


Si lo miras de esta manera, P> 0.05, entonces tenemos que aceptar la hipótesis nula, lo que significa que este lote de leche cumple con el estándar y no se mezcla con agua.Pero esta pregunta es en realidad una prueba unilateral, Lo que debemos considerar es solo la parte de la leche cuya temperatura es más alta que la temperatura del punto de congelación, mientras que la leche cuya temperatura es más baja que la temperatura del punto de congelación es leche normal y no contiene agua.

Cuanto más a la izquierda, más se acepta la hipótesis nula, lo que se necesita es que el valor P esté cerca de 1 cuando el área de la derecha está en el extremo izquierdo.

    Ahora veremos intervalos de confianza pero sacare las definiciones de internet dado que no estan bien los metodos del libro

In [224]:
import numpy as np 
from scipy.stats import t

Let’s now simulate a dataset made of 100 numbers extracted from a normal distribution.

In [238]:
x = np.random.normal(size=100)

In [232]:
x = [1,2,3,1,2,3,1,2,4,5,2,4,5,2,4,6,2,4,1,2,3,4]


2.8636363636363638

Let’s see we want to calculate the 95% confidence interval of the mean value. Let’s calculate all the numbers we need according to the formula of confidence intervals.

In [239]:
m = np.mean(x)
#desviasion estandar
s = np.std(x) 
dof = len(x)-1 
confidence = 0.95

We now need the value of t. The function that calculates the inverse cumulative distribution is ppf. We need to apply the absolute value because the cumulative distribution works with the left tail, so the result would be negative

In [240]:
t_crit = np.abs(t.ppf((1-confidence)/2,dof))

Now, we can apply the original formula to calculate the 95% confidence interval.

In [242]:
(m-s*t_crit/np.sqrt(len(x)), m+s*t_crit/np.sqrt(len(x))) 
# (-0.14017768797464097, 0.259793719043611)

(-0.37221797102164283, 0.055143011885080584)

We know it’s correct because the normal distribution has 0 mean, but if we don’t know anything about the population, we could say that, with 95% confidence, the expected value of the population lies between -0.14 and 0.26.
We could have reached the same result using a bootstrap, which is unbiased. In this example, I create 1000 resamples of our dataset (with replacements).

In [237]:
values = [np.random.choice(x,size=len(x),replace=True).mean() for i in range(1000)] 
np.percentile(values,[100*(1-confidence)/2,100*(1-(1-confidence)/2)]) 
# array([-0.13559955, 0.26480175])

array([2.27272727, 3.45454545])

In [248]:
## P HACKING

def run_experiment():
    #Aventar una moneda mil veces
    return[np.random.random()<0.5 for _ in range(1000)]

def reject_fairness(experiment):
    #usar el 5% de significancia
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531
np.random.seed(9)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

print (num_rejections)

#una prueba de hipotesis non es sustituto de el sentido común

66


## Corriendo una A/B Test

    Vamos a correr una prueba basada en dos tipos de comerciales para una bebida energetica desarrollada para Cientificos de Datos, mientras que la primera "most datafulllly drink" // y la segunda "datum data balum"
    

In [253]:
def estimated_parameters(N,n):
    p = n / N
    sigma = math.sqrt(p * (1 -p)/N)
    return p,sigma

In [254]:
def a_b_test_stadistic(N_A,n_A,N_B,n_B):
    p_A, sigma_A = estimated_parameters(N_A,n_A)
    p_B, sigma_B = estimated_parameters(N_B,n_B)
    return(p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

Ahora si la estrategia A obtiene 200 clicks de 1,000 vistas y la estrategia B obtiene 180 de 1,000 vistas, la estadistica equivale a : 

In [256]:
z = a_b_test_stadistic(1000,200,1000,180)
z

-1.1403464899034472

Buscando ese valor en una tabla de distribucion normal en : https://www.socscistatistics.com/pvalues/normaldistribution.aspx <br>
Con la configuracion de two tailed nos da como valor <br><br>
##### The P-Value is $  0.254286.  $

Esto significa que no hay mucha diferencia entre una campaña y otra y se puede continuar con ambas, sin ningnun cambio signficativo

#### Ahora veremos otra prueba de A/B Testing donde:

Problema de muestra : Identifique si un correo promocional de mejor apariencia lograría que más clientes se inscribieran en un club de entrega que cuesta $100.

A fines de junio, enviamos por correo contenido promocional a algunos de nuestros clientes para nuestro "Club de entrega". Inscribirse en el club cuesta dólares y les brindamos a los clientes entregas de comestibles gratuitas durante un año, a partir del 1 de julio.<br>
Queríamos ver si un anuncio publicitario de mejor apariencia lograría que más clientes se suscribieran. Seleccionamos al azar 3 grupos, el primer grupo recibió el Mailer 1, una versión muy básica y económica que describía el trato, el segundo grupo recibió el Mailer 2, un mailer muy bonito y colorido impreso en cartón, y el tercer grupo fue el grupo de control y recibido al correo en absoluto.<br>
Estamos muy seguros de que los clientes a los que se envió por correo se inscribieron a una tasa mucho más alta que el grupo de control... pero no estamos seguros de si existe una diferencia significativa entre la tasa de registro para el correo más barato y el más caro. . A primera vista, parece haber una tasa de registro ligeramente más alta para la versión costosa, pero antes de sacar conclusiones, queríamos que revisara los números.<br>
### La tarea
Evaluar si existe una diferencia entre el correo 1 y el correo 2 en términos de tasa de inscripción al club <br>
### Las Hipótesis <br>
null_hypothesis = No existe una relación entre el tipo de correo y la tasa de suscripción y son independientes.
alternate_hypothesis = Existe una relación entre el tipo de remitente y la tasa de suscripción y que no son independientes.
### La prueba utilizada <br>
#### Prueba de chi-cuadrado
##### El resultado
chi2_estadística = 1,94<p>
p_valor = 0,16<p>
valor_critico = 3.84<p>
Se mantiene la hipótesis nula. No existe una relación entre el tipo de correo y la tasa de registro. son independientes

In [261]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt

# import chi2_contingency to compute for the chi-square statistics and the p-value for the particular hypothesis test
# import chi2 to find the critical value based on the acceptance criteria
from scipy.stats import chi2_contingency, chi2 

# import data
campaign_data = pd.read_excel("./recursos/grocery_database.xlsx", sheet_name = "campaign_data")

#Obtener frecuencias observadas

# create a 2x2 matrix array
# summarise to get our observed frequencies
observed_values = pd.crosstab(campaign_data["mailer_type"], campaign_data["signup_flag"]).values
print(observed_values)

mailer1_signup_rate = 123 / (252 + 123) 
mailer2_signup_rate = 127 / (209 + 127)
print(mailer1_signup_rate, mailer2_signup_rate)

[[140  19]
 [252 123]
 [209 127]]
0.328 0.37797619047619047


In [259]:
# state hypotheses & set acceptance criteria
null_hypothesis = "There is no relationship between mailer type and signup rate. They are independent."
alternate_hypothesis = "There is a relationship between mailer type and signup rate. They are not independent."
acceptance_criteria = 0.05

In [262]:
# calculate expected frequencies & chi square statistic
chi2_statistic, p_value, dof, expected_values = chi2_contingency(observed_values, correction = False)
print(chi2_statistic, p_value)

# find the critical value for our test
critical_value = chi2.ppf(1 - acceptance_criteria, dof)
print(critical_value)

34.85054342506307 2.7058308848098242e-08
5.991464547107979


### Resumen de los resultados<p>
El problema de muestra había evaluado la diferencia en la tasa de inscripción al club entre dos correos diferentes que se enviaron. Y en base a los datos proporcionados y usando la prueba de chi-cuadrado para la independencia, ahora puedo recomendar a ABC Grocery y tal vez a su departamento de marketing que pueden dejar de enviar un anuncio por correo de apariencia costosa o incluso un anuncio por correo simple para ahorrar costos, ya que no ayuda. en lograr que los clientes se inscriban en la promoción del club de entrega.